<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2024-08-24 13:47:19--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  7.51MB/s    in 25s     

2024-08-24 13:47:45 (9.87 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

2024-08-24 13:47:57.942314: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 13:47:57.984280: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-24 13:47:57.984315: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 13:47:57.984345: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-24 13:47:57.992207: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 13:47:57.993320: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
# Parameters
NUM_CLASSES = 2
IMAGE_RESIZE = 224
BATCH_SIZE = 100

In [5]:
# Data generation with preprocessing specific to VGG16
data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
)

# Training data generator
training_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size = (IMAGE_RESIZE, IMAGE_RESIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True,
)

# Validation data generator
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (IMAGE_RESIZE, IMAGE_RESIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = False,
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [6]:
# build VGG16 model
model_vgg16 = Sequential()
model_vgg16.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    )
)
model_vgg16.add(Dense(NUM_CLASSES, activation='softmax'))
model_vgg16.layers[0].trainable = False
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg16.summary()

2024-08-24 13:48:01.414479: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


58889256/58889256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [7]:
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

# Train the model
fit_history = model_vgg16.fit_generator(
    training_generator,
    steps_per_epoch = steps_per_epoch_training,
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = steps_per_epoch_validation,
    verbose = 1,
)

# Save model
model_vgg16.save('classifier_vgg16_model.h5')

/tmp/ipykernel_887/4142478533.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model_vgg16.fit_generator(


Epoch 1/2
301/301 [==============================] - 975s 3s/step - loss: 0.1088 - accuracy: 0.9627 - val_loss: 0.0254 - val_accuracy: 0.9938
Epoch 2/2
301/301 [==============================] - 975s 3s/step - loss: 0.0194 - accuracy: 0.9956 - val_loss: 0.0152 - val_accuracy: 0.9955


/opt/conda/envs/myenv/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [8]:
# Load the RESNET50 model from the module 3
model_resnet50 = load_model('classifier_resnet_model.h5')

In [9]:
# Test data generator
evaluate_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(IMAGE_RESIZE, IMAGE_RESIZE),
    class_mode='categorical',
    shuffle=False
)

# Eval the ResNet50 model
resnet50_results = model_resnet50.evaluate(evaluate_generator, verbose=1)
resnet50_loss, resnet50_accuracy = resnet50_results[0], resnet50_results[1]

# Eval the VGG16 model
vgg16_results = model_vgg16.evaluate(evaluate_generator, verbose=1)
vgg16_loss, vgg16_accuracy = vgg16_results[0], vgg16_results[1]

print(f"ResNet50 Model - Loss: {resnet50_loss:.4f}, Accuracy: {resnet50_accuracy:.4f}")
print(f"VGG16 Model - Loss: {vgg16_loss:.4f}, Accuracy: {vgg16_accuracy:.4f}")

Found 500 images belonging to 2 classes.
16/16 [==============================] - 13s 797ms/step - loss: 0.0148 - accuracy: 0.9960
ResNet50 Model - Loss: 0.0096, Accuracy: 0.9960
VGG16 Model - Loss: 0.0148, Accuracy: 0.9960


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [10]:
# Predict classes using the ResNet50 model
resnet50_predictions = model_resnet50.predict(evaluate_generator)
# Predict classes using the VGG16 model
vgg16_predictions = model_vgg16.predict(evaluate_generator)

# Get class indices to map predictions to class labels
class_indices = evaluate_generator.class_indices
inverse_class_indices = {v: k for k, v in class_indices.items()}  # Invert the dictionary

# Convert predictions to class labels (assuming 'Positive' is class 1 and 'Negative' is class 0)
resnet50_predicted_classes = np.argmax(resnet50_predictions, axis=1)
vgg16_predicted_classes = np.argmax(vgg16_predictions, axis=1)

# Convert numeric predictions to class names
resnet50_predicted_labels = [inverse_class_indices[i] for i in resnet50_predicted_classes]
vgg16_predicted_labels = [inverse_class_indices[i] for i in vgg16_predicted_classes]

# Display the first five predictions for ResNet50
print("ResNet50 Model - First Five Predictions:")
for i in range(5):
    print("Positive" if resnet50_predicted_labels[i] == 'positive' else "Negative")

# Display the first five predictions for VGG16
print("\nVGG16 Model - First Five Predictions:")
for i in range(5):
    print("Positive" if vgg16_predicted_labels[i] == 'positive' else "Negative")


16/16 [==============================] - 13s 814ms/step
ResNet50 Model - First Five Predictions:
Negative
Negative
Negative
Negative
Negative

VGG16 Model - First Five Predictions:
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).